In [1]:
import time
import pyrealsense2 as rs
import numpy as np
import open3d as o3d
import json
import cv2

In [2]:
# Load json file
json_path = '/home/commaai-03/Test/test.json'
with open(json_path, 'r') as f:
    json_obj = json.load(f)
    
json_str = str(json_obj).replace("'", '\"')

In [3]:
ctx = rs.context()                                                
devices = ctx.query_devices()                                     
#print('devices: ', devices)
advance_mode = rs.rs400_advanced_mode(devices[0])
advance_mode.load_json(json_str)

In [4]:
points = [[453, 223], [340, 223], [24, 256], [767, 437], [828, 13]]

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848, 480, rs.format.rgb8, 30)

pipeline.start(config)

spatial = rs.spatial_filter()
temporal = rs.temporal_filter()

# skip some frames
for x in range(30):
    pipeline.wait_for_frames()

distances = [[0] for i in range(len(points))]

for i in range(1):
    align_to = rs.stream.color
    align = rs.align(align_to)

    frames = pipeline.wait_for_frames()
    aligned_frames = align.process(frames)

    color = aligned_frames.get_color_frame()
    depth = aligned_frames.get_depth_frame()
    
    #depth = decimation.process(depth)
    depth = spatial.process(depth)
    depth = temporal.process(depth)
    

    d_img = np.asanyarray(depth.get_data())
    #d_img_1 = nonZeroSubsampling(d_img, kernel_size=2, method='mean')
    img = np.asanyarray(color.get_data())
    print(img.shape)

'''
    for i in range(len(points)):
        x, y = points[i]
        dic = d_img[y, x].astype(float)
        distances[i].append(dic)

        
distances = np.asarray(distances)
distances = distances[:, 1:]
std = np.std(distances, axis=1)
std
'''

(480, 848, 3)


'\n    for i in range(len(points)):\n        x, y = points[i]\n        dic = d_img[y, x].astype(float)\n        distances[i].append(dic)\n\n        \ndistances = np.asarray(distances)\ndistances = distances[:, 1:]\nstd = np.std(distances, axis=1)\nstd\n'

In [6]:
d = pipeline.wait_for_frames().get_depth_frame()
dpt_frame = d.as_depth_frame()
pixel_distance_in_meters = dpt_frame.get_distance(233,455)
pixel_distance_in_meters

2.182000160217285

In [ ]:
profiles = pipeline.get_active_profile()

In [ ]:
profile = profiles.get_stream(rs.stream.color).as_video_stream_profile()

In [ ]:
profile.get_intrinsics()

In [ ]:
data = d_img.reshape([240, 2, 424, 2])
data = np.transpose(data, [0, 2, 1, 3])
data = data.reshape(240, 424, 4)
'''
mean = np.true_divide(np.sum(data, axis=(2,3)), 
                      np.count_nonzero(data, axis=(2, 3)))
mean = np.nan_to_num(mean)
mean
'''

In [ ]:
np.mean(, axis=2)

In [ ]:
data.reshape(240, 424, 4)

In [ ]:
np.median(data[np.nonzero(data)], axis=[2,3])

In [ ]:
x, y = points[0]
dic = d_img[x, y]
dic.astype(float)

In [ ]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848, 480, rs.format.rgb8, 30)

pipeline.start(config)

# skip some frames
for x in range(99):
    pipeline.wait_for_frames()

align_to = rs.stream.color
align = rs.align(align_to)
add = False

# Filter
decimation = rs.decimation_filter()
spatial = rs.spatial_filter()
temporal = rs.temporal_filter()

try:
    vis = o3d.visualization.Visualizer()
    vis.create_window('D435', width=1280, height=720)
    pcd = o3d.geometry.PointCloud()
    while 1:
        t0 = time.time()
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        profile = aligned_frames.get_profile()
        color = aligned_frames.get_color_frame()
        depth = aligned_frames.get_depth_frame()
        #depth = decimation.process(depth)
        #depth = spatial.process(depth)
        #depth = temporal.process(depth)
        if not color or not depth:
            continue
        
        d_img = np.asanyarray(depth.get_data())
        d_img = nonZeroSubsampling(d_img, kernel_size=2, method='mean')
        img = np.asanyarray(color.get_data())
        img = cv2.resize(img, (424, 240))
        
        d_img = o3d.geometry.Image(d_img)
        img = o3d.geometry.Image(img)
        
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(img, d_img, depth_trunc=5.0, convert_rgb_to_intensity=False)
        intrinsics = profile.as_video_stream_profile().get_intrinsics()
        pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(intrinsics.width, intrinsics.height, intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy)
        pc = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, pinhole_camera_intrinsic)
        pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
        pcd.points = pc.points
        pcd.colors = pc.colors
        
        if not add:
            vis.add_geometry(pcd)
            add = True
            
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        t1 = time.time()
        print('[FPS]: %.2f' % (1/(t1 - t0)))
finally:
    pipeline.stop()
    vis.destroy_window()

In [ ]:
?rs.decimation_filter

In [ ]:
import pyrealsense2 as rs
import time
import json

DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07","0B3A"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

try:
    dev = find_device_that_supports_advanced_mode()
    advnc_mode = rs.rs400_advanced_mode(dev)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Loop until we successfully enable advanced mode
    while not advnc_mode.is_enabled():
        print("Trying to enable advanced mode...")
        advnc_mode.toggle_advanced_mode(True)
        # At this point the device will disconnect and re-connect.
        print("Sleeping for 5 seconds...")
        time.sleep(5)
        # The 'dev' object will become invalid and we need to initialize it again
        dev = find_device_that_supports_advanced_mode()
        advnc_mode = rs.rs400_advanced_mode(dev)
        print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Get each control's current value
    print("Depth Control: \n", advnc_mode.get_depth_control())
    print("RSM: \n", advnc_mode.get_rsm())
    print("RAU Support Vector Control: \n", advnc_mode.get_rau_support_vector_control())
    print("Color Control: \n", advnc_mode.get_color_control())
    print("RAU Thresholds Control: \n", advnc_mode.get_rau_thresholds_control())
    print("SLO Color Thresholds Control: \n", advnc_mode.get_slo_color_thresholds_control())
    print("SLO Penalty Control: \n", advnc_mode.get_slo_penalty_control())
    print("HDAD: \n", advnc_mode.get_hdad())
    print("Color Correction: \n", advnc_mode.get_color_correction())
    print("Depth Table: \n", advnc_mode.get_depth_table())
    print("Auto Exposure Control: \n", advnc_mode.get_ae_control())
    print("Census: \n", advnc_mode.get_census())

    #To get the minimum and maximum value of each control use the mode value:
    query_min_values_mode = 1
    query_max_values_mode = 2
    current_std_depth_control_group = advnc_mode.get_depth_control()
    min_std_depth_control_group = advnc_mode.get_depth_control(query_min_values_mode)
    max_std_depth_control_group = advnc_mode.get_depth_control(query_max_values_mode)
    print("Depth Control Min Values: \n ", min_std_depth_control_group)
    print("Depth Control Max Values: \n ", max_std_depth_control_group)

    # Set some control with a new (median) value
    current_std_depth_control_group.scoreThreshA = int((max_std_depth_control_group.scoreThreshA - min_std_depth_control_group.scoreThreshA) / 2)
    advnc_mode.set_depth_control(current_std_depth_control_group)
    print("After Setting new value, Depth Control: \n", advnc_mode.get_depth_control())

    # Serialize all controls to a Json string
    serialized_string = advnc_mode.serialize_json()
    print("Controls as JSON: \n", serialized_string)
    as_json_object = json.loads(serialized_string)

    # We can also load controls from a json string
    # For Python 2, the values in 'as_json_object' dict need to be converted from unicode object to utf-8
    if type(next(iter(as_json_object))) != str:
        as_json_object = {k.encode('utf-8'): v.encode("utf-8") for k, v in as_json_object.items()}
    # The C++ JSON parser requires double-quotes for the json object so we need
    # to replace the single quote of the pythonic json to double-quotes
    json_string = str(as_json_object).replace("'", '\"')
    advnc_mode.load_json(json_string)

except Exception as e:
    print(e)
    pass

In [ ]:
# Sub-sampling

import numpy as np

def nonZeroSubsampling(data, kernel_size=1, method='median'):
    '''Non-zero median or Non-zero mean
    Args:
     - data: input depth data
     - kerne_size: 
     - method: median or mean
    '''
    if not type(data) == np.ndarray:
        data = np.asarray(data)
    assert len(data.shape) == 2
    assert isinstance(kernel_size, int) or (isinstance(kernel_size, tuple) and len(kernel_size)==2)
    assert method.lower() in ['median', 'mean']
    if isinstance(kernel_size, int):
        stride_x, stride_y = kernel_size, kernel_size
    else:
        stride_x, stride_y = kernel_size
        
    assert data.shape[0] % stride_x == 0 and data.shape[1] % stride_y == 0
    
    shape = [data.shape[0] // stride_x, stride_x, data.shape[1] // stride_y, stride_y]
    data = data.reshape(shape)
    data = np.transpose(data, [0, 2, 1, 3])
    
    if method.lower() == 'mean':
        mean = np.true_divide(np.sum(data, axis=(2,3)), 
                      np.count_nonzero(data, axis=(2, 3)))
        mean = np.nan_to_num(mean)
        return mean.astype(np.uint16)
    
    subsampling = np.zeros([shape[0], shape[2]])
    
    for i in range(shape[0]):
        for j in range(shape[2]):
            window = data[i, j]
            non_zero = window[np.nonzero(window)]
            subsampling[i, j] = np.median(non_zero)
            subsampling = np.nan_to_num(subsampling)
                
    return subsampling.astype(np.uint16)

In [ ]:
# temporal filtering

class TemporalFilter():
    
    def __init__(self, alpha=0.5, delta=20, initiation=True):
        self.alpha = alpha
        self.delta = delta
        self.initiation = initiation
        self.source = None
    
    def process(self, x):
        assert type(x) == np.ndarray and len(x.shape == 2)
        if self.initiation:
            self.source = x
            self.initiation = False
            return x
        
        target = self.alpha * x + (1 - self.alpha) * self.source
        print(target)
        thresh = np.abs(target - self.source)
        self.source = np.where(thresh < self.delta, 
                               target, x)
        return self.source